## SENTIMENT ANALYSIS

In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
covid_ph_df = pd.read_csv("../datasets/covid_philippines/filtered_comments.csv").drop("Unnamed: 0", axis=1)
covid_ph_df.head()

,video_id,comment,comment_cleaned
0,aLZ85hb4wjE,Loved the calmness of Manilla.,loved the calmness of manilla.
1,aLZ85hb4wjE,After this pandemic I think every country shou...,after this pandemic i think every country shou...
2,aLZ85hb4wjE,manila looks beautifull with less people,manila looks beautifull with less peopl
3,aLZ85hb4wjE,The lockdown makes the city look like a place ...,the lockdown makes the city look like a place ...
4,aLZ85hb4wjE,India also same,india also sam


In [3]:
covid_vax_df = pd.read_csv("../datasets/covid_vaccine/filtered_comments.csv").drop("Unnamed: 0", axis=1)
covid_vax_df.head()

,video_id,comment,comment_cleaned
0,2IXl4qJGrRk,That is one Guiness Book of records that will ...,that is one guiness book of records that will ...
1,2IXl4qJGrRk,He's now the strongest man in the world.,he's now the strongest man in the world.
2,2IXl4qJGrRk,He got more vaccines than all Texans together!,he got more vaccines than all texans together!
3,2IXl4qJGrRk,this man is more vaccine than he is human,this man is more vaccine than he is human
4,2IXl4qJGrRk,That’s the difference between Germany and Amer...,that’s the difference between germany and amer...


In [4]:
is_pal_df = pd.read_csv("../datasets/israel-palestine_conflict_history/filtered_comments.csv").drop("Unnamed: 0", axis=1)
is_pal_df.head()

,video_id,comment,comment_cleaned
0,R0ftmf_Uv9A,No matter how many times these information get...,no matter how many times these information get...
1,R0ftmf_Uv9A,Let peace prevail.,let peace prevail.
2,R0ftmf_Uv9A,Why start at 1946?,why start at 1946?
3,R0ftmf_Uv9A,What happened the video was removed and put ag...,what happened the video was removed and put ag...
4,R0ftmf_Uv9A,"This is not a conflict, this is occupation.","this is not a conflict, this is occupation."


TEST RoBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
from scipy.special import softmax

In [193]:


def roberta_sentiment_scores(list_comments):
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    labels = ['Negative', 'Neutral', 'Positive']

    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)

    for comments in list_comments:
        comment_words = []
        comments = comments.replace("\n", " ")
        comments = comments.replace("\xa0", " ")
        comments = comments.replace("?", " ")
        comments = comments.replace(":", " ")
        comments = comments.replace(";", " ")
        comments = comments.replace(";", " ")
        comments = re.sub(r"\s+", ' ', comments) 
        print(comments)
        for word in comments.split(' '):
            if word.startswith('@') and len(word) > 1:
               word = '@user'
        
            elif word.startswith('http'):
                word = "http"
            comment_words.append(word)

        comment_procs = " ".join(comment_words)

        encoded = tokenizer(comment_procs, return_tensors='pt', max_length=512, truncation=True, padding=True)
        print(encoded)
        output = model(**encoded)

        scores = output[0][0].detach().numpy()

        scores = softmax(scores)

        for i in range(len(scores)):

            l = labels[i]
            s = scores[i]
            print(l, s)

In [194]:
roberta_sentiment_scores(covid_vax_filtered["comment"][:50])

d:\Program Files\miniconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


That is one Guiness Book of records that will never fall.
{'input_ids': tensor([[   0, 1711,   16,   65, 2646, 6601, 5972,    9, 2189,   14,   40,  393,
         1136,    4,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Negative 0.02012999
Neutral 0.4440781
Positive 0.53579193
He's now the strongest man in the world.
{'input_ids': tensor([[   0,  894,   18,  122,    5, 8260,  313,   11,    5,  232,    4,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Negative 0.0020073105
Neutral 0.030271364
Positive 0.9677213
He got more vaccines than all Texans together!
{'input_ids': tensor([[    0,   894,   300,    55, 16968,    87,    70, 10818,   561,   328,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Negative 0.4294948
Neutral 0.47008806
Positive 0.100417115
this man is more vaccine than he is human
{'input_ids': tensor([[   0, 9226,  313,   16,   55, 9937,   87,   37,   16, 1050,    2]]), 'at

TEST TEXTBLOB

In [195]:
from textblob import TextBlob

def textblob_sentiment_scores(list_comments):
    list_sentiment={}
    for comment in list_comments:
        testimonial = TextBlob(comment)
        if (testimonial.sentiment.polarity > 0):
            print("positive", testimonial.sentiment.polarity)
        elif (testimonial.sentiment.polarity < 0):
            print("negative", testimonial.sentiment.polarity)
        else:
            print("neutral", testimonial.sentiment.polarity)

In [196]:
textblob_sentiment_scores(covid_vax_filtered["comment"][:50])

neutral 0.0
neutral 0.0
positive 0.625
positive 0.25
neutral 0.0
positive 0.625
positive 0.2
neutral 0.0
positive 0.125
positive 0.6
negative -0.05
neutral 0.0
negative -0.3
neutral 0.0
positive 0.625
neutral 0.0
neutral 0.0
negative -0.42500000000000004
positive 0.39285714285714285
neutral 0.0
positive 0.3
positive 0.2857142857142857
neutral 0.0
negative -0.30833333333333335
neutral 0.0
negative -0.08333333333333333
positive 0.2765151515151515
neutral 0.0
positive 0.08124999999999999
neutral 0.0
negative -0.23437500000000003
neutral 0.0
negative -0.07698412698412699
neutral 0.0
positive 0.3571428571428571
neutral 0.0
positive 0.6
negative -0.0625
neutral 0.0
positive 0.2778409090909091
positive 0.2
neutral 0.0
negative -0.3125
neutral 0.0
positive 0.16666666666666666
neutral 0.0
neutral 0.0
positive 0.4375
negative -0.55
neutral 0.0


TEST STANZA

In [197]:
import stanza
def stanza_sentiment_scores(list_comments):
    nlp = stanza.Pipeline('en', processors='tokenize,sentiment', tokenize_no_ssplit=True)

    for comment in list_comments:
        doc = nlp(comment.replace("\n", " "))
        print(comment)
    #doc.sentences[0].print_dependencies()
        for i, sentence in enumerate(doc.sentences):
            print("%d -> %d" % (i, sentence.sentiment))

In [198]:
stanza_sentiment_scores(covid_vax_filtered["comment"][:50])

2024-03-15 18:28:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-15 18:28:49 WARNING: Language en package default expects mwt, which has been added
2024-03-15 18:28:51 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| sentiment | sstplus  |

2024-03-15 18:28:51 INFO: Using device: cpu
2024-03-15 18:28:51 INFO: Loading: tokenize
2024-03-15 18:28:51 INFO: Loading: mwt
2024-03-15 18:28:51 INFO: Loading: sentiment
2024-03-15 18:28:52 INFO: Done loading processors!


That is one Guiness Book of records that will never fall.
0 -> 2
He's now the strongest man in the world.
0 -> 2
He got more vaccines than all Texans together!
0 -> 0
this man is more vaccine than he is human
0 -> 1
That’s the difference between Germany and America: man receives 217 Covid shots vs man gives 217 gunshots.
0 -> 1
I would love to hear the response from vaccine sceptics who predicted the end of time for people who accepted one vaccine!
0 -> 0
Doesn't he know you need 218 vaccinations to be really protected. 😂 🐑
0 -> 1
There are two factors helping your immunity with a vaccine. Temporary heightened immune response to all infections and later an immune response to the specific infectious agent.
0 -> 2
Wow, this dude’s 5g signal must be off the charts!
0 -> 1
It's good to know the vaccine is safe to use.
0 -> 2
Strange kink. Though not gonna judge him for it.
0 -> 1
no proof, no identity, just words.
0 -> 0
In todays late news..He exploded 3 minutes after this footage was doc

TEST VADER

In [199]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def vader_sentiment_scores (list_comments):
    for sentence in list_comments:
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(sentence)
        print(sentence)
        print("Overall sentiment dictionary is : ", sentiment_dict)
        print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
        print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
        print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
 
        print("Sentence Overall Rated As", end = " ")
 
        # decide sentiment as positive, negative and neutral
        if sentiment_dict['compound'] >= 0.05 :
            print("Positive")
 
        elif sentiment_dict['compound'] <= - 0.05 :
            print("Negative")
 
        else :
            print("Neutral")

    

In [200]:
vader_sentiment_scores (covid_vax_filtered["comment"][:50])

That is one Guiness Book of records that will never fall.
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
sentence was rated as  0.0 % Negative
sentence was rated as  100.0 % Neutral
sentence was rated as  0.0 % Positive
Sentence Overall Rated As Neutral
He's now the strongest man in the world.
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.707, 'pos': 0.293, 'compound': 0.4404}
sentence was rated as  0.0 % Negative
sentence was rated as  70.7 % Neutral
sentence was rated as  29.299999999999997 % Positive
Sentence Overall Rated As Positive
He got more vaccines than all Texans together!
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
sentence was rated as  0.0 % Negative
sentence was rated as  100.0 % Neutral
sentence was rated as  0.0 % Positive
Sentence Overall Rated As Neutral
this man is more vaccine than he is human
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'com

Doesn't he know you need 218 vaccinations to be really protected. 😂 🐑
Overall sentiment dictionary is :  {'neg': 0.083, 'neu': 0.612, 'pos': 0.305, 'compound': 0.7264}
sentence was rated as  8.3 % Negative
sentence was rated as  61.199999999999996 % Neutral
sentence was rated as  30.5 % Positive
Sentence Overall Rated As Positive
There are two factors helping your immunity with a vaccine. Temporary heightened immune response to all infections and later an immune response to the specific infectious agent.
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.6808}
sentence was rated as  0.0 % Negative
sentence was rated as  78.4 % Neutral
sentence was rated as  21.6 % Positive
Sentence Overall Rated As Positive
Wow, this dude’s 5g signal must be off the charts!
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'compound': 0.6239}
sentence was rated as  0.0 % Negative
sentence was rated as  68.7 % Neutral
sentence was rated as

### Get Sentiment sentiment for each video

#### TEXTBLOB APPROACH

In [201]:
def textblob_sentiment_scores(list_comments):
    list_sentiment={}
    for comment in list_comments:
        testimonial = TextBlob(comment)
        if (testimonial.sentiment.polarity > 0):
            print("positive", testimonial.sentiment.polarity)
        elif (testimonial.sentiment.polarity < 0):
            print("negative", testimonial.sentiment.polarity)
        else:
            print("neutral", testimonial.sentiment.polarity)

In [202]:
covid_vax_filtered["video_id"].unique()

array(['2IXl4qJGrRk', 'HtTalpY-J-M', 'jPs4_MeuX7U', 'WhiBpmH1mE4',
       'LfmhYVCCGhc', 'oFDOQ9Pn-XM', '-Ea3UwYZnrA', 'im3otpqYAiQ',
       'geXKovieb1E', 'm2skKkRwYj8', 'YVFAvoeFahI', 'IMRTSwb3hCs',
       'mkZ1MabMEZw', 'sidjKoHS6NE', 'xbYYNpM1AgQ', 'KBvys-kcJi0',
       'P3nXJB5PoBM', '85ZicYaZsbQ', 'SkcAZfrYYXM', 'I9S2jooTxkQ',
       'NGxPkc_9uhY', 'uOxppT3gPpE', '3rIQHrbDguM', 'fdGj6p2qDdk',
       'rNaIMC4XMhc', 'vilMNHn4Yh8', 'Ptrblo96DEI', 'xnTVis4scUc',
       'UijQ-6jhVuc', 'nqZdzaKFF3c', 'r62K2ZzOEko', 'uZPGZ2mf4g4',
       'Z-zWo835gOs', 'Y0sg9G5BBVU', '40eZeXPyJ0g', 'xYW-o8CNQiU',
       'eFx-d0SvOVg', 'i9JLZuhqRuU', '6mH0_S-_FqE', 'Li-oxKhvZy0',
       '3GVGd3Ckrxk', 'l84zftrMoyc', 'aGKPq9OUEP0', 'EETuOY3JjfM',
       'DM_71oPxEhE', 'Jb2YMvfvm_M', 'AQhiMirOODY', '1m_iclAMXLo',
       'Kd-E95tXfsY', '-92HQA0GcI8', 'RaLxhFiOBYk', 'sv0dQfRRrEQ',
       'aOInguXymGc', 'lEFYYpE_hds', 'v-NEr3KCug8', 'IxCHv7T_mao',
       'qVcP3cI64ow', 'iyo2UNQcdpQ', 'iMx7kmld9sk', 'oMXGGmBfk

In [222]:
def per_vid_textblob_polarity (df):
    video_polarity={}
    for video in df["video_id"].unique():
        sum_polarity=0
        for comment in df.loc[df["video_id"]==video]["comment"]:
            sum_polarity+=(TextBlob(comment)).sentiment.polarity
        video_polarity[video] = [sum_polarity/df.loc[df["video_id"]==video]["comment"].count()]
    ''' if (testimonial.sentiment.polarity > 0):
            print("positive", testimonial.sentiment.subjectivity)
        elif (testimonial.sentiment.polarity < 0):
            print("negative", testimonial.sentiment.subjectivity)
        else:
            print("neutral", testimonial.sentiment.subjectivity)'''
    
    return video_polarity


In [1]:
df= pd.DataFrame.from_dict(per_vid_textblob_polarity(covid_vax_filtered))


NameError: name 'pd' is not defined

#### VADER APPROACH

In [206]:
def per_vid_vader_polarity (df):
    polarity_scores={}
    sid_obj = SentimentIntensityAnalyzer()

    for video in df["video_id"].unique():
        sum_neg =0
        sum_pos = 0
        sum_neu =0
        sum_compound= 0
        scores={}
        for sentence in df.loc[df["video_id"]==video]["comment"]:

            sentiment_dict = sid_obj.polarity_scores(sentence)
            sum_neg += sentiment_dict['neg']
            sum_pos += sentiment_dict['pos']
            sum_neu += sentiment_dict['neu']
            sum_compound += sentiment_dict['compound']
            
        scores["Neg"] = sum_neg/df.loc[df["video_id"]==video]["comment"].count()
        scores["Pos"] = sum_pos/df.loc[df["video_id"]==video]["comment"].count()
        scores["Neu"] = sum_neu/df.loc[df["video_id"]==video]["comment"].count()
        scores["Overall"] = sum_compound/df.loc[df["video_id"]==video]["comment"].count()
        
        polarity_scores[video]=scores
        '''# decide sentiment as positive, negative and neutral
        if sentiment_dict['compound'] >= 0.05 :
            print("Positive")
 
        elif sentiment_dict['compound'] <= - 0.05 :
            print("Negative")
 
        else :
            print("Neutral")'''
    return polarity_scores
        

In [207]:
per_vid_vader_polarity(covid_vax_filtered)

{'2IXl4qJGrRk': {'Neg': 0.016111111111111114,
  'Pos': 0.15211111111111109,
  'Neu': 0.8317777777777777,
  'Overall': 0.35336666666666666},
 'HtTalpY-J-M': {'Neg': 0.10566666666666666,
  'Pos': 0.16555555555555557,
  'Neu': 0.7286666666666667,
  'Overall': 0.1677777777777778},
 'jPs4_MeuX7U': {'Neg': 0.22014285714285714,
  'Pos': 0.12842857142857142,
  'Neu': 0.6512857142857145,
  'Overall': -0.18910000000000002},
 'WhiBpmH1mE4': {'Neg': 0.03325,
  'Pos': 0.16237500000000002,
  'Neu': 0.8045,
  'Overall': 0.3346625},
 'LfmhYVCCGhc': {'Neg': 0.108,
  'Pos': 0.04485714285714286,
  'Neu': 0.8472857142857142,
  'Overall': -0.12254285714285713},
 'oFDOQ9Pn-XM': {'Neg': 0.16469999999999999,
  'Pos': 0.1377,
  'Neu': 0.6976,
  'Overall': 0.05447},
 '-Ea3UwYZnrA': {'Neg': 0.05025,
  'Pos': 0.196875,
  'Neu': 0.75275,
  'Overall': 0.35605000000000003},
 'im3otpqYAiQ': {'Neg': 0.0828,
  'Pos': 0.1638,
  'Neu': 0.7535000000000001,
  'Overall': 0.05452},
 'geXKovieb1E': {'Neg': 0.11811111111111111

### RoBERTa APPROACH

In [208]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
#import re
#from scipy.special import softmax

def roberta_per_vid_scores(df):
    polarity_scores={}

    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    labels = ['Negative', 'Neutral', 'Positive']

    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)
    for video in df["video_id"].unique():
        sum_neg =0
        sum_pos = 0
        sum_neu =0 
        scores={}
        for comments in df.loc[df["video_id"]==video]["comment"]:
            comment_words = []
            comments = comments.replace("\n", " ")
            comments = comments.replace("\xa0", " ")
            comments = comments.replace("?", " ")
            comments = comments.replace(":", " ")
            comments = comments.replace(";", " ")
            comments = comments.replace(";", " ")
            comments = re.sub(r"\s+", ' ', comments) 
     #   print(comments)
            for word in comments.split(' '):
                if word.startswith('@') and len(word) > 1:
                    word = '@user'
        
                elif word.startswith('http'):
                    word = "http"
                comment_words.append(word)

            comment_procs = " ".join(comment_words)

            encoded = tokenizer(comment_procs, return_tensors='pt', max_length=512, truncation=True, padding=True)
           # print(encoded)
            output = model(**encoded)

            scores = output[0][0].detach().numpy()

            scores = softmax(scores)

            for i in range(len(scores)):

                l = labels[i]
                s = scores[i]
                #    labels = ['Negative', 'Neutral', 'Positive']

                if l=="Negative":
                    sum_neg+= s
                elif l == "Neutral":
                    sum_neu+=s
                else : 
                    sum_pos+=s
       # scores["Neg"] = sum_neg/df.loc[df["video_id"]==video]["comment"].count()
        scores =dict([("Neg", sum_neg/df.loc[df["video_id"]==video]["comment"].count()),("Pos",sum_pos/df.loc[df["video_id"]==video]["comment"].count()),("Neu",sum_neu/df.loc[df["video_id"]==video]["comment"].count()
)])
        #scores["Pos"] = sum_pos/df.loc[df["video_id"]==video]["comment"].count()
        #scores["Neu"] = sum_neu/df.loc[df["video_id"]==video]["comment"].count()

        polarity_scores[video] = scores
    return polarity_scores

In [209]:
roberta_per_vid_scores(covid_vax_filtered)

{'2IXl4qJGrRk': {'Neg': 0.26506742930764127,
  'Pos': 0.3514234812464565,
  'Neu': 0.3835090897563431},
 'HtTalpY-J-M': {'Neg': 0.3412811907215251,
  'Pos': 0.2002104783637656,
  'Neu': 0.45850832511981326},
 'jPs4_MeuX7U': {'Neg': 0.5579070512737546,
  'Pos': 0.15677219968555228,
  'Neu': 0.28532079234719276},
 'WhiBpmH1mE4': {'Neg': 0.17986852729518432,
  'Pos': 0.5050607317825779,
  'Neu': 0.3150707244640216},
 'LfmhYVCCGhc': {'Neg': 0.48226415259497507,
  'Pos': 0.1623721340937274,
  'Neu': 0.35536372874464306},
 'oFDOQ9Pn-XM': {'Neg': 0.5234494373202324,
  'Pos': 0.10175626734271645,
  'Neu': 0.3747942989692092},
 '-Ea3UwYZnrA': {'Neg': 0.3052443841879722,
  'Pos': 0.45912257861346006,
  'Neu': 0.23563304590061307},
 'im3otpqYAiQ': {'Neg': 0.49974274933338164,
  'Pos': 0.13261141991242767,
  'Neu': 0.36764583326876166},
 'geXKovieb1E': {'Neg': 0.5631933104660776,
  'Pos': 0.19209164665598008,
  'Neu': 0.24471505317423078},
 'm2skKkRwYj8': {'Neg': 0.6514680369291455,
  'Pos': 0.154